<a href="https://colab.research.google.com/github/ShreyasJothish/DS-Unit-4-Sprint-1-Tree-Ensembles/blob/master/module2-random-forests/DS1_Tree_Ensembles_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [2]:
# You also have to join the Titanic competition to have access to the data
!kaggle competitions download -c ds1-tree-ensembles

train_features.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_labels.csv: Skipping, found more recently modified local copy (use --force to force download)
test_features.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
LCDataDictionary.xlsx: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
#!unzip train_features.csv.zip
#!unzip test_features.csv.zip

In [4]:
!pip install category_encoders

In [0]:
# Generic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
X_train = pd.read_csv("train_features.csv")
y_train = pd.read_csv("train_labels.csv")['charged_off']
print(X_train.shape)
print(y_train.shape)

(37745, 103)
(37745,)


In [0]:
pd.set_option('display.max_columns', None)  # Unlimited columns
pd.set_option('display.max_rows', None)  # Unlimited rows

In [8]:
X_train.isnull().sum()

id                                         0
member_id                              37745
loan_amnt                                  0
funded_amnt                                0
term                                       0
int_rate                                   0
installment                                0
grade                                      0
sub_grade                                  0
emp_title                               3565
emp_length                              3277
home_ownership                             0
annual_inc                                 0
url                                    37745
desc                                   37745
purpose                                    0
title                                      0
zip_code                                   0
addr_state                                 0
dti                                       92
delinq_2yrs                                0
earliest_cr_line                           0
inq_last_6

In [9]:
X_train.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
0,43373,NaN,12000,12000,36 months,16.02%,422.01,C,C5,Driver,4 years,MORTGAGE,81000.0,NaN,NaN,debt_consolidation,Debt consolidation,280xx,NC,12.76,0,Dec-2005,1,48.0,NaN,7,0,4912,23.5%,15,w,0,48.0,Individual,NaN,NaN,0,0,174783,1,2,0,4,13.0,33668,82.0,2,5,2962,63.0,20900,1,1,2,9,24969.0,588.0,89.3,0,0,145.0,23,1,1,1,8.0,NaN,1.0,NaN,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.0,0,0,206618,38580,5500,40863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,24414,NaN,6300,6300,36 months,14.07%,215.54,C,C3,GMP Lead,4 years,RENT,39000.0,NaN,NaN,debt_consolidation,Debt consolidation,920xx,CA,21.42,0,Mar-2012,1,NaN,NaN,10,0,11876,59.4%,12,w,0,NaN,Individual,NaN,NaN,0,0,31046,0,1,0,2,14.0,19170,83.0,3,8,5153,72.0,20000,1,0,1,10,3105.0,6877.0,59.3,0,0,17.0,72,8,8,0,8.0,NaN,6.0,NaN,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.9,0,0,43140,31046,16900,23140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,46723,NaN,4500,4500,36 months,7.21%,139.38,A,A3,Accounts,3 years,RENT,78000.0,NaN,NaN,debt_consolidation,Debt consolidation,906xx,CA,2.17,0,May-2000,2,NaN,NaN,13,0,1715,5.2%,19,w,0,NaN,Individual,NaN,NaN,0,0,35329,0,8,1,2,10.0,33614,106.0,1,1,1196,54.0,33300,4,0,3,3,2718.0,21585.0,7.4,0,0,166.0,218,10,10,0,31.0,NaN,1.0,NaN,0,3,3,3,4,13,5,6,3,13,0.0,0,0,2,100.0,0.0,0,0,65092,35329,23300,31792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,24878,NaN,12000,12000,36 months,9.44%,384.06,B,B1,Client Processing Associate,10+ years,MORTGAGE,62000.0,NaN,NaN,debt_consolidation,Debt consolidation,441xx,OH,13.76,0,Feb-2003,0,NaN,NaN,22,0,9404,16.3%,33,w,0,NaN,Individual,NaN,NaN,0,0,75160,4,1,0,2,13.0,17123,67.0,6,9,6898,32.0,57600,3,1,2,12,3579.0,33602.0,17.0,0,0,162.0,179,1,1,1,1.0,NaN,5.0,NaN,0,1,3,9,13,7,20,25,3,22,0.0,0,0,7,100.0,0.0,0,0,133065,26527,40500,25465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,39038,NaN,12000,12000,36 months,22.35%,460.47,D,D5,Construction Operator,10+ years,MORTGAGE,70000.0,NaN,NaN,debt_consolidation,Debt consolidation,751xx,TX,14.04,4,Nov-2001,2,13.0,NaN,18,0,17895,24.2%,45,w,0,13.0,Individual,NaN,NaN,0,7218,113702,9,2,2,4,6.0,16711,68.0,9,10,5373,35.0,73900,3,6,3,14,6317.0,32863.0,24.1,0,0,159.0,201,0,0,1,0.0,62.0,3.0,62.0,6,9,11,11,14,22,15,20,11,18,0.0,0,2,11,82.2,9.1,0,0,192842,34606,43300,24499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash


In [10]:
"""
member_id                              37745
emp_title                               3565
emp_length                              3277
url                                    37745
desc                                   37745
dti                                       92
mths_since_last_delinq                 20881
mths_since_last_record                 32170
revol_util                                53
mths_since_last_major_derog            28640
annual_inc_joint                       33007
dti_joint                              33007
mths_since_rcnt_il                      1234
il_util                                 6255
all_util                                  13
avg_cur_bal                                3
bc_open_to_buy                           643
bc_util                                  667
mo_sin_old_il_acct                      1234
mths_since_recent_bc                     597
mths_since_recent_bc_dlq               29995
mths_since_recent_inq                   3146
mths_since_recent_revol_delinq         26574
num_tl_120dpd_2m                        1010
pct_tl_nvr_dlq                             1
percent_bc_gt_75                         643
revol_bal_joint                        33007
sec_app_earliest_cr_line               33007
sec_app_inq_last_6mths                 33007
sec_app_mort_acc                       33007
sec_app_open_acc                       33007
sec_app_revol_util                     33106
sec_app_open_act_il                    33007
sec_app_num_rev_accts                  33007
sec_app_chargeoff_within_12_mths       33007
sec_app_collections_12_mths_ex_med     33007
sec_app_mths_since_last_major_derog    36077
"""

'\nmember_id                              37745\nemp_title                               3565\nemp_length                              3277\nurl                                    37745\ndesc                                   37745\ndti                                       92\nmths_since_last_delinq                 20881\nmths_since_last_record                 32170\nrevol_util                                53\nmths_since_last_major_derog            28640\nannual_inc_joint                       33007\ndti_joint                              33007\nmths_since_rcnt_il                      1234\nil_util                                 6255\nall_util                                  13\navg_cur_bal                                3\nbc_open_to_buy                           643\nbc_util                                  667\nmo_sin_old_il_acct                      1234\nmths_since_recent_bc                     597\nmths_since_recent_bc_dlq               29995\nmths_since_recent_inq          

In [0]:
# Feature Engineering

def feature_engineering(df_input):
  
  df = df_input.copy()
  
  df.drop(columns=['id',
                'member_id',              
                'url',                                
                'desc',                               
                'mths_since_last_delinq',             
                'mths_since_last_record',             
                'mths_since_last_major_derog',        
                'annual_inc_joint',                   
                'dti_joint',                          
                'il_util',                            
                'mths_since_recent_bc_dlq',           
                'mths_since_recent_revol_delinq',     
                'revol_bal_joint',                    
                'sec_app_earliest_cr_line',           
                'sec_app_inq_last_6mths',             
                'sec_app_mort_acc',                   
                'sec_app_open_acc',                   
                'sec_app_revol_util',                 
                'sec_app_open_act_il',                
                'sec_app_num_rev_accts',              
                'sec_app_chargeoff_within_12_mths',   
                'sec_app_collections_12_mths_ex_med', 
                'sec_app_mths_since_last_major_derog',
                'num_tl_120dpd_2m', # No variance
                'emp_title',        # 16270 unique values
                #'zip_code',         # 855 unique values
                #'earliest_cr_line', # 596 unique values
               ], inplace=True)
  
  # term	- Convert to int
  def term_to_int(term_str):
    return int(term_str.replace(" months",""))
  
  df['term'] = df['term'].apply(term_to_int)

  # int_rate - Convert to float
  def int_rate_to_float(int_rate_str):
    return float(int_rate_str.replace("%",""))
  
  df['int_rate'] = df['int_rate'].apply(int_rate_to_float)

  # emp_length - Fill NA as Unknown and use encoding
  df['emp_length'].fillna("Unknown", inplace=True)

  # dti - Fill NA as 0.00
  df['dti'].fillna(0.00, inplace=True)

  # revol_util - Convert to float
  df['revol_util'].fillna("0.00%", inplace=True)

  def revol_util_to_float(revol_util_str):
    return float(revol_util_str.replace("%",""))

  df['revol_util'] = df['revol_util'].apply(revol_util_to_float)


  # mths_since_rcnt_il - Fill NA with mean
  df['mths_since_rcnt_il'].fillna(df['mths_since_rcnt_il'].mean(), 
                                     inplace=True)


  # all_util - Fill NA with mean
  df['all_util'].fillna(df['all_util'].mean(), inplace=True)


  # bc_open_to_buy - Fill NA with mean
  df['bc_open_to_buy'].fillna(df['bc_open_to_buy'].median(), 
                                 inplace=True)

  # bc_util - Fill NA with mean
  df['bc_util'].fillna(df['bc_util'].mean(), inplace=True)

  # mo_sin_old_il_acct - Fill NA with mean
  df['mo_sin_old_il_acct'].fillna(df['mo_sin_old_il_acct'].mean(), 
                                     inplace=True)

  # mths_since_recent_bc - Fill NA with mean
  df['mths_since_recent_bc'].fillna(df['mths_since_recent_bc'].mean(), 
                                 inplace=True)

  # mths_since_recent_inq - Fill NA with mean
  df['mths_since_recent_inq'].fillna(df['mths_since_recent_inq'].mean(), 
                                  inplace=True)

  # avg_cur_bal - Fill NA with mean
  df['avg_cur_bal'].fillna(df['avg_cur_bal'].mean(), 
                                  inplace=True)

  # pct_tl_nvr_dlq - Fill NA with mean
  df['pct_tl_nvr_dlq'].fillna(df['pct_tl_nvr_dlq'].mean(), 
                                  inplace=True)

  # percent_bc_gt_75 - Fill NA with mean
  df['percent_bc_gt_75'].fillna(df['percent_bc_gt_75'].mean(), 
                                  inplace=True)
  
  return df

In [0]:
X_train = feature_engineering(X_train)

In [13]:
import category_encoders as ce
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

"""pipeline = make_pipeline(
    ce.BinaryEncoder(), 
    DecisionTreeClassifier(max_depth=5)
)"""

pipeline = make_pipeline(
    ce.BinaryEncoder(), 
    RandomForestClassifier(
        n_estimators=100, 
        class_weight='balanced', 
        min_samples_leaf=0.005, 
        oob_score=True, 
        n_jobs=-1)
)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('binaryencoder', BinaryEncoder(cols=['grade', 'sub_grade', 'emp_length', 'home_ownership', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'initial_list_status', 'application_type', 'disbursement_method'],
       drop_invariant=False, handle_unknown='impute', impute_missing...mators=100, n_jobs=-1, oob_score=True, random_state=None,
            verbose=0, warm_start=False))])

In [14]:
# Out-of-Bag estimated score
from sklearn.metrics import roc_auc_score

y_pred_proba = pipeline.named_steps['randomforestclassifier'].oob_decision_function_[:, 1]
print('ROC AUC, Out-of-Bag estimate:', roc_auc_score(y_train, y_pred_proba))

ROC AUC, Out-of-Bag estimate: 0.7235478109944842


In [0]:
X_test = pd.read_csv("test_features.csv")
X_test = feature_engineering(X_test)

In [16]:
# Use GridSearch CV for hyper parameter tuning

from sklearn.model_selection import GridSearchCV

param_grid = {
    'randomforestclassifier__n_estimators': [400, 500, 600],
    'randomforestclassifier__min_samples_leaf': [0.001, 0.002, 0.005]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3,
                         scoring='roc_auc', verbose=10)

gridsearch.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] randomforestclassifier__min_samples_leaf=0.001, randomforestclassifier__n_estimators=400 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  randomforestclassifier__min_samples_leaf=0.001, randomforestclassifier__n_estimators=400, score=0.7323670969615695, total=  30.2s
[CV] randomforestclassifier__min_samples_leaf=0.001, randomforestclassifier__n_estimators=400 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   32.6s remaining:    0.0s


[CV]  randomforestclassifier__min_samples_leaf=0.001, randomforestclassifier__n_estimators=400, score=0.7284591135594322, total=  27.9s
[CV] randomforestclassifier__min_samples_leaf=0.001, randomforestclassifier__n_estimators=400 


...

[CV]  randomforestclassifier__min_samples_leaf=0.001, randomforestclassifier__n_estimators=600, score=0.7405760368315557, total=  42.3s
[CV] randomforestclassifier__min_samples_leaf=0.002, randomforestclassifier__n_estimators=400 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  5.6min remaining:    0.0s


[CV]  randomforestclassifier__min_samples_leaf=0.002, randomforestclassifier__n_estimators=400, score=0.7297814329732011, total=  25.9s
[CV] randomforestclassifier__min_samples_leaf=0.002, randomforestclassifier__n_estimators=400 
[CV]  randomforestclassifier__min_samples_leaf=0.002, randomforestclassifier__n_estimators=400, score=0.7248891433209323, total=  25.0s
[CV] randomforestclassifier__min_samples_leaf=0.002, randomforestclassifier__n_estimators=400 
[CV]  randomforestclassifier__min_samples_leaf=0.002, randomforestclassifier__n_estimators=400, score=0.7387704084792331, total=  24.9s
[CV] randomforestclassifier__min_samples_leaf=0.002, randomforestclassifier__n_estimators=500 
[CV]  randomforestclassifier__min_samples_leaf=0.002, randomforestclassifier__n_estimators=500, score=0.7297308135952982, total=  31.1s
[CV] randomforestclassifier__min_samples_leaf=0.002, randomforestclassifier__n_estimators=500 
[CV]  randomforestclassifier__min_samples_leaf=0.002, randomforestclassifier

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 14.9min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('binaryencoder', BinaryEncoder(cols=['grade', 'sub_grade', 'emp_length', 'home_ownership', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'initial_list_status', 'application_type', 'disbursement_method'],
       drop_invariant=False, handle_unknown='impute', impute_missing...mators=100, n_jobs=-1, oob_score=True, random_state=None,
            verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'randomforestclassifier__n_estimators': [400, 500, 600], 'randomforestclassifier__min_samples_leaf': [0.001, 0.002, 0.005]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [17]:
# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Cross Validation Score: 0.7343492903308948
Best Parameters: {'randomforestclassifier__min_samples_leaf': 0.001, 'randomforestclassifier__n_estimators': 600}


In [0]:
sample_submission = pd.read_csv('sample_submission.csv') 
submission = sample_submission.copy() 
submission['charged_off'] = gridsearch.predict_proba(X_test)[:, 1] 
submission.to_csv('submission-007.csv', index=False)

In [20]:
!pip install eli5

    100% |████████████████████████████████| 102kB 2.7MB/s 


In [23]:
import eli5
from eli5.sklearn import PermutationImportance

encoder = ce.BinaryEncoder()
X_train_transformed = encoder.fit_transform(X_train)

model = RandomForestClassifier(
    n_estimators=600, 
    class_weight='balanced', 
    min_samples_leaf=0.001, 
    n_jobs=-1)

model.fit(X_train_transformed, y_train)
permuter = PermutationImportance(model, scoring='roc_auc', n_iter=3, cv='prefit')
permuter.fit(X_train_transformed, y_train)

PermutationImportance(cv='prefit',
           estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=0.001,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=600, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False),
           n_iter=3, random_state=None, refit=True, scoring='roc_auc')

In [24]:
eli5.show_weights(permuter, top=None, feature_names=X_train_transformed.columns.tolist())

Weight,Feature
0.0314 ± 0.0017,int_rate
0.0170 ± 0.0010,installment
0.0164 ± 0.0007,dti
0.0152 ± 0.0005,funded_amnt
0.0148 ± 0.0009,loan_amnt
0.0134 ± 0.0009,tot_hi_cred_lim
0.0118 ± 0.0005,tot_cur_bal
0.0113 ± 0.0006,avg_cur_bal
0.0113 ± 0.0012,grade_2
0.0108 ± 0.0004,num_il_tl
